In [40]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [41]:
name_city = "Hamburg"
number_of_rooms = "1,00"
nb_pages = 1
#Get number of pages with results
req = requests.get("https://www.immobilienscout24.de/Suche/S-T/P-1/Wohnung-Kauf/%s/%s/-/-%s" 
                   % (name_city, name_city, number_of_rooms))
content = req.content
pars = BeautifulSoup(content, "html.parser")
try:
    div_nb_pages = pars.find("div", {"id":"pageSelection"}).find("select", {"class":"select font-standard"}).find_all("option")
    pages_list = []
    for x in div_nb_pages:
        pages_list.append(x.text)
    if len(pages_list) > 1:
        nb_pages = max(pages_list)
except:
    pass

In [42]:
list_districts_as_str = []
price_for_one_sq_m = []

for i in pages_list:
    # Lets load the web pages
    # https://www.immobilienscout24.de/Suche/S-T/P-1/Wohnung-Kauf/Hamburg/Hamburg/-/-1,00 
    # /P-1/ -> is a current page
    req = requests.get("https://www.immobilienscout24.de/Suche/S-T/P-%s/Wohnung-Kauf/%s/%s/-/-%s" 
                   % (i, name_city, name_city, number_of_rooms))
    content = req.content
    
    pars = BeautifulSoup(content, "html.parser")
    all_divs = pars.find_all("div", {"class":"grid grid-flex gutter-horizontal-l gutter-vertical-s"})
    divs_districts = pars.find_all("div", {"class":"result-list-entry__address"})
    
    pattern = "\d+\.?\d+"
    
    # get all prices as strings from page
    all_prices_as_str = []
    for price in all_divs:
        temp = price.find_all("dd", {"class":"font-line-xs"})[0].text[0:-2]
        temp = temp.replace('.' , '')
        all_prices_as_str.append(temp)

    # convert prices into numeric
    all_prices_as_float = []
    for price in all_prices_as_str:
        if len(re.findall(pattern, price)) > 1:
            temp = re.findall(pattern, price)
            temp = (int(temp[0]) + int(temp[1])) / 2
            all_prices_as_float.append(temp)
        else:
            all_prices_as_float.append(int(re.findall(pattern, price)[0]))
    del all_prices_as_str

    # get all living spaces as strings from page
    all_living_spaces_as_str = []
    for space in all_divs:
        s = space.find_all("dd", {"class":"font-line-xs"})[1].text[0:-3]
        s = s.replace(',' , '.')
        all_living_spaces_as_str.append(s)

    # convert all living spaces into numeric
    living_spaces_as_float = []
    for p in all_living_spaces_as_str:
        if len(re.findall(pattern, p)) > 1:
            temp = re.findall(pattern, p)
            temp = (float(temp[0]) + float(temp[1])) / 2
            living_spaces_as_float.append(temp)
        else:
            living_spaces_as_float.append(float(re.findall(pattern, p)[0]))
    del all_living_spaces_as_str

    # get all districts as strings from page
    list_districts = []
    for district in divs_districts:
        temp = district.find_all("div", {"class":"font-ellipsis"})[0].text
        list_districts.append(temp)

    # convert districts into region of city
    for address in list_districts:
        temp = address.split(',')
        if len(temp) > 2:
            res = temp[1][1:]
        else:
            res = temp[0]
        list_districts_as_str.append(res)
    del list_districts

    # calculate the average price
    for x, y in zip(all_prices_as_float, living_spaces_as_float):
        price_for_one_sq_m.append(x / y)
    del living_spaces_as_float, all_prices_as_float
    
# Group average prices by regions of city
df = {'District' : list_districts_as_str, 'Average_Price': price_for_one_sq_m}
data_frame = pd.DataFrame(data = df).groupby('District').mean().reset_index()

In [43]:
data_frame

,District,Average_Price
0,Alsterdorf,6488.404638
1,Altona-Altstadt,6030.631780
2,Bahrenfeld,6421.194153
3,Barmbek-Nord,3515.625000
4,Barmbek-Süd,6395.833333
5,Bergedorf,3634.134177
6,Billstedt,2978.884079
7,Bramfeld,3212.996390
8,Eidelstedt,3128.073185
9,Eilbek,4324.943439
